# Simulation of emittance measurement

In [ ]:
import RF_Track as rft 
import numpy as np 
import scipy
import matplotlib.pyplot as plt 

# Bunch parameters

In [ ]:
# Bunch parameters
mass = rft.electronmass # MeV / c^2
charge=-1 # single particle charge, in units of e
population = 1 * rft.nC # number of real particles per bunch
Pref = 5 # reference momentum, MeV / c
P_Q = Pref / charge # MV / c, reference rigidity

# Creating an essential beam line

In [ ]:
Lcell=2 # m
Lquad=0.2 # m
Ldrift=Lcell/2-Lquad # m
L=0.5*Lquad+Ldrift
mu = np.deg2rad(90) # deg
K1L=-np.sin(mu/2) / (Lcell/4) # 1/m

strength=K1L * P_Q # MeV / m

# A full, focusing quadrupole
Q=rft.Quadrupole(Lquad,strength)

# The Drift
Dr = rft.Drift(Ldrift)
Dr.set_tt_nsteps(100)

# The Screen

S = rft.Screen()

# Defining the sequence

Beamline=rft.Lattice()
Beamline.append(Q)
Beamline.append(Dr)
Beamline.append(S)

# Creating a bunch and specifying the Twiss parameters and emittance

In [ ]:
Twiss=rft.Bunch6d_twiss()
Twiss.beta_x = Lcell * (1 + np.sin(mu/2)) / np.sin(mu) # m
Twiss.beta_y = Lcell * (1 - np.sin(mu/2)) / np.sin(mu) # m
Twiss.alpha_x = 0.0
Twiss.alpha_y = 0.0
Twiss.emitt_x = 0.3 # mm.mrad, normalized emittances
Twiss.emitt_y = 0.005 # mm.mrad

N = 10000
B0 = rft.Bunch6d(mass, population, charge, Pref, Twiss, N)

# Track the bunch through the lattice

Beamline.track(B0)


# Calculating beam sizes and getting transport table

Beam size is obtained from:
\begin{equation*}
\sigma_x = \sqrt{\langle x^2\rangle - \langle x\rangle^2},
\qquad
\sigma_y = \sqrt{\langle y^2\rangle - \langle y\rangle^2}.
\end{equation*}






In [ ]:

T = Beamline.get_transport_table('%S %sigma_x %sigma_y %sigma_t %sigma_E %mean_E %emitt_x %emitt_y %beta_x %beta_y')

B=Beamline.get_bunch_at_screens()

M=B[0].get_phase_space('%x %y')
x,y = M[:,0], M[:,1]

# Calculate beam sizes

sigma_x= np.sqrt(np.mean(x**2) - (np.mean(x))**2)
sigma_y= np.sqrt(np.mean(y**2) - (np.mean(y))**2)

print("Beam sizes for nominal quadrupole settings are: ")
print(f"Sigma x: {sigma_x} mm")
print(f"Sigma y: {sigma_y} mm")

# Performing quadrupole scan by varying its strength and fitting the parameters

The result is fitted with a parabola. One parametrization for the fit is:
\begin{equation*}
\Sigma_{11} \;=\; A\,(K - B)^2 + C =\; a\,K^2 + b\,K + c.
\end{equation*}
And then:
\begin{aligned}
A = a,\quad B = -\frac{b}{2a},\quad C = c - \frac{b^2}{4a}.\\[4pt]
\end{aligned}

Standard error of beam size calculations:
\begin{equation*}
\mathrm{SE}\!\left[s^{2}\right] \;=\; \sigma^{2}\,\sqrt{\frac{2}{\,N-1\,}} \, .
\end{equation*}


In [ ]:
K1L_values=np.linspace(-1/L-3/L,-1/L+3/L,25)
sigma_x2_values,sigma_y2_values = [], []

for K1L_value in K1L_values:
    Beamline[0].set_K1L(abs(P_Q), K1L_value)
    Beamline.track(B0)
    M = Beamline.get_bunch_at_screens()[0].get_phase_space('%x %y')
    x,y = M[:,0],M[:,1]
    sigma_x2_values.append(np.var(x)) # mm^2
    sigma_y2_values.append(np.var(y)) # mm^2
sigma_x2_errors=[value * np.sqrt(2/(N-1)) for value in sigma_x2_values]
sigma_y2_errors=[value * np.sqrt(2/(N-1)) for value in sigma_y2_values]
# Fitting a parabola in plane u (x or y)

a_x,b_x,c_x=np.polyfit(K1L_values,sigma_x2_values,2)
A_x = a_x
B_x = -b_x/(2*a_x)
C_x = c_x-(b_x*b_x/(4*a_x))

a_y,b_y,c_y=np.polyfit(K1L_values,sigma_y2_values,2)
A_y = a_y
B_y = -b_y/(2*a_y)
C_y = c_y-(b_y*b_y/(4*a_y))

print("Plane x: ")
print(f"A = {A_x} ")
print(f"B = {B_x} ")
print(f"C = {C_x}")

print("Plane y: ")
print(f"A = {A_y} ")
print(f"B = {B_y} ")
print(f"C = {C_y}")

The total transfer matrix of interest here is R = SQ, where S denotes the known transfer matrix between the quadrupole and the wire, and Q is the transfer matrix of the quadrupole:
\begin{equation*}
\\Q = \left(\begin{array}{rl}
\mathbf{1}&\mathbf{0}\\
\mathbf{K} & \mathbf{1}\\
\end{array}\right),
\end{equation*}

With 
\begin{equation*}
\\S = R_{drift} = \left(\begin{array}{rl}
\mathbf{1}&\mathbf{L}\\
\mathbf{0} & \mathbf{1}\\
\end{array}\right),
\end{equation*}

Which gives us:

\begin{equation*}
\\R = \left(\begin{array}{rl}
\mathbf{S_{11}+KS_{12}}&\mathbf{S_{12}}\\
\mathbf{S_{21}+KS_{22}} & \mathbf{S_{22}}\\
\end{array}\right)=
\left(\begin{array}{rl}
\mathbf{1+KL}&\mathbf{L}\\
\mathbf{K} & \mathbf{1}\\
\end{array}\right),
\end{equation*}




# Simulation of emittance measurement and calculating elements of a beam matrix

The beam matrix is reconstructed by equating coefficients of:
\begin{equation*}
{\Sigma_{11}(K)}
=\big(S_{11}^2\,\Sigma_{11,0}+2S_{11}S_{12}\,\Sigma_{12,0}+S_{12}^2\,\Sigma_{22,0}\big)
+\big(2S_{11}S_{12}\,\Sigma_{11,0}+2S_{12}^2\,\Sigma_{12,0}\big)\,K
+\big(S_{12}^2\,\Sigma_{11,0}\big)\,K^2 ,
\end{equation*}

which is quadratic in the field parameter K, and:
\begin{equation*}
\Sigma_{11}(K)=A\,(K-B)^2+C \;=\; A K^2 - 2 A B K + (C + A B^2).
\end{equation*}

Resulting in:
\begin{align*}
\Sigma_{11} &= \frac{A}{S_{12}^{2}},\\[4pt]
\Sigma_{12} &= -\,\frac{A}{S_{12}^{2}}\!\left(B+\frac{S_{11}}{S_{12}}\right),\\[4pt]
\Sigma_{22} &= \frac{(A B^{2}+C)+2AB\left(\frac{S_{11}}{S_{12}}\right)+A\left(\frac{S_{11}}{S_{12}}\right)^{2}}{S_{12}^{2}}.
\end{align*}




In [ ]:
sigma_x_11 = A_x / (L**2)
sigma_x_12 = (-A_x/(L**2)) * (B_x + 1 / L)
sigma_x_22 = (1 / (L**2)) * (( A_x * (B_x**2) + C_x) + 2 * A_x * B_x* (1 / L) + A_x * (1 / L)**2)

sigma_y_11 = A_y / (L**2)
sigma_y_12 = (-A_y/(L**2)) * (B_y + 1 / L)
sigma_y_22 = (1 / (L**2)) * (( A_y * (B_y**2) + C_y) + 2 * A_y * B_y* (1 / L) + A_y * (1 / L)**2)


The beam emittance is then calculated from the determinant of the beam matrix \begin{equation*}
\varepsilon_x \;=\; \sqrt{\det\Sigma_x} \;=\; \sqrt{\Sigma_{11}\Sigma_{22}-\Sigma_{12}^{\,2}} \, .
\end{equation*}
 and the errors are propagated:


In [ ]:
det_sigma_x_beam = sigma_x_11 * sigma_x_22 - sigma_x_12**2
det_sigma_y_beam = sigma_y_11 * sigma_y_22 - sigma_y_12**2

emittance_x = np.sqrt(det_sigma_x_beam)
emittance_y = np.sqrt(det_sigma_y_beam)
print(f"Emittance in the plane x: {emittance_x} mm.rad")
print(f"Emittance in the plane y: {emittance_y} mm.rad")

# Alternative way
# det_sigma_x_beam = A_x * C_x  / (L**4)
# det_sigma_y_beam = A_x * C_x  / (L**4)

# emittance_x = np.sqrt(A_y * C_y) / (L**2)
# emittance_y = np.sqrt(A_y * C_y) / (L**2)

Calculating magnetic rigidity:
\begin{equation*}
B\rho
\;=\; 3.356\,p[\mathrm{GeV}/c]\;\mathrm{T\cdot m}
\;=\; 10\times 3.356\,p[\mathrm{GeV}/c]\;\mathrm{kG\cdot m}.
\end{equation*}

\begin{equation*}
\Delta\!\Big(\!\int G\,ds\Big)\;[\mathrm{kG}]
\;=\; \big(B\rho\;[\mathrm{kG\cdot m}]\big)\;\Delta\!\big(K_{1}L\;[\mathrm{m^{-1}}]\big).
\end{equation*}

\begin{equation*}
\text{x–plane axis: }\quad
G^{(x)}_{\!ds} \;=\; B\rho\,\big(K_{1}L - B_x\big),
\qquad
\text{y–plane axis: }\quad
G^{(y)}_{\!ds} \;=\; B\rho\,\big(K_{1}L - B_y\big).
\end{equation*}

In [ ]:
# Magnetic rigidity of the beam
B_rho = 3.356 * (Pref/1000)*10 #kG * m
K1L0 = K1L_values[len(K1L_values)//2]
# Change in the quadrupole field
G_ds_x = B_rho * (K1L_values - B_x) #kG
G_ds_y = B_rho * (K1L_values - B_y)


# Time for plots!

In [ ]:
plt.figure()
plt.errorbar(G_ds_x, sigma_x2_values, yerr=sigma_x2_errors, fmt='o', capsize=3, label=r'data: $\sigma_x^2$')
ax,bx,cx = np.polyfit(G_ds_x, np.array(sigma_x2_values), 2)
xf = np.linspace(G_ds_x.min(), G_ds_x.max(), 400)
plt.plot(xf, ax*xf**2 + bx*xf + cx, '-', label='quadratic fit')
plt.xlabel('change in quadrupole field [kG]'); plt.ylabel(r'$\sigma_x^2$ [mm$^2$]')
plt.legend(); plt.tight_layout(); plt.show()

plt.figure()
plt.errorbar(G_ds_y, sigma_y2_values, yerr=sigma_y2_errors, fmt='o', capsize=3, label=r'data: $\sigma_y^2$')
ay,by,cy = np.polyfit(G_ds_y, np.array(sigma_y2_values), 2)
yf = np.linspace(G_ds_y.min(), G_ds_y.max(), 400)
plt.plot(yf, ay*yf**2 + by*yf + cy, '-', label='quadratic fit')
plt.xlabel('change in quadrupole field [kG]'); plt.ylabel(r'$\sigma_y^2$ [mm$^2$]')
plt.legend(); plt.tight_layout(); plt.show()

The above results also give the beam-ellipse parameters and another equation for calculating emittance: \begin{equation*}
\varepsilon \;=\; \frac{\sqrt{A\,C}}{S_{12}^{2}} \,,
\qquad
\beta_x \;=\; \frac{\Sigma_{11}}{\varepsilon}
\;=\; \sqrt{\frac{A}{C}} \,,
\qquad
\alpha_x \;=\; -\,\frac{\Sigma_{12}}{\varepsilon}
\;=\; \sqrt{\frac{A}{C}}\!\left( B + \frac{S_{11}}{S_{12}} \right),
\end{equation*}
\begin{equation*}
\gamma_x \;=\; \frac{S_{12}^{2}}{\sqrt{A\,C}}
\left[\,(A B^{2}+C)\;+\;2 A B\,(\frac{S_{11}}{S_{12}})
\;+\;A\!\left(\frac{S_{11}}{S_{12}}\right)^{2}\right].
\end{equation*}



In [ ]:
# Emittance
emitt_x = np.sqrt(A_x*C_x) / L**2
emitt_y = np.sqrt(A_y*C_y) / L**2
print(f"Emittance in the plane x: {emitt_x} mm.rad")
print(f"Emittance in the plane y: {emitt_y} mm.rad")

# Beta
beta_x = sigma_x_11/emitt_x
beta_y = sigma_y_11/emitt_y
print(f"Beta in the plane x: {beta_x}")
print(f"Beta in the plane y: {beta_y}")

# Alpha
alpha_x = -sigma_x_12/emitt_x
alpha_y = -sigma_y_12/emitt_y
print(f"Alpha in the plane x: {alpha_x}")
print(f"Alpha in the plane y: {alpha_y}")

# Gamma
gamma_x = sigma_x_22/emitt_x
gamma_y = sigma_y_22/emitt_y

print(f"Gamma in the plane x: {gamma_x}")
print(f"Gamma in the plane y: {gamma_y}")

As a useful check, the beam-ellipse parameters should satisfy:
\begin{equation*}
\beta_x\,\gamma_x \;-\; 1 \, = \alpha_x^{2}.
\end{equation*}




In [ ]:
# Left side
print("x check:", beta_x*gamma_x - 1 - alpha_x**2)
print("y check:", beta_y*gamma_y - 1 - alpha_y**2)

Checks should be near 0!